# Day 7: Amplification Circuit
This day will be a lot like Day 5, but with a few modifications.

Import `itertools` to get all permutations of a list.

In [1]:
import itertools

In [2]:
with open('day_07.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [3]:
# Opcode IDs
ADD = 1
MULTIPLY = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6
LESS_THAN = 7
EQUALS = 8
HALT = 99

In [4]:
OP_NAMES = {
    ADD: 'Add',
    MULTIPLY: 'Multiply',
    INPUT: 'Input',
    OUTPUT: 'Output',
    JUMP_IF_TRUE: 'Jump If True',
    JUMP_IF_FALSE: 'Jump If False',
    LESS_THAN: 'Less Than',
    EQUALS: 'Equals',
    HALT: 'Halt',
}

In [5]:
# Opcode param lengths
NUM_PARAMS = {
    ADD: 3,
    MULTIPLY: 3,
    INPUT: 1,
    OUTPUT: 1,
    JUMP_IF_TRUE: 2,
    JUMP_IF_FALSE: 2,
    LESS_THAN: 3,
    EQUALS: 3,
    HALT: 0,
}

In [6]:
# Opcode param modes
POSITION_MODE = 0
IMMEDIATE_MODE = 1

In [7]:
ACTION_NEXT = 0
ACTION_JUMP = 1
ACTION_OUTPUT = 2
ACTION_HALT = 3

Here we are modifying the `Instruction` class by adding a `return_control` method, which returns a tuple of `action` and `value`. This will let the caller know what to do next:
* `ACTION_NEXT` tells the caller to move the instruction pointer to the next sequential instruction (`value` is `None`)
* `ACTION_JUMP` tells the caller to set the instruction pointer to the provided `value`
* `ACTION_OUTPUT` tells the caller to move the instruction pointer to the next sequential instruction, pause execution, and return the provided `value` to the caller's caller (lol)
* `ACTION_HALT` tells the caller to halt entirely

In [8]:
class Instruction:
    def __init__(self, opcode, length, params, modes, memory):
        self.opcode = opcode
        self.length = length
        self.params = params
        self.modes = modes
        self.memory = memory
        
    def get_val(self, param_index):
        if self.modes[param_index] == IMMEDIATE_MODE:
            return self.params[param_index]
        elif self.modes[param_index] == POSITION_MODE: 
            return self.memory[self.params[param_index]]
    
    def put_val(self, param_address_index, value):
        self.memory[self.params[param_address_index]] = value
        
    def return_control(self, action = ACTION_NEXT, value = None):
        return [action, value]

To account for this new `return_control()` paradigm, the `call()` method of each subclass of `Instruction` will now ultimately call `return_control()` itself.

In [9]:
class Add(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 + val1)
        return self.return_control()

In [10]:
class Multiply(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        self.put_val(2, val0 * val1)
        return self.return_control()                     

The `Input` instruction's `call()` method has been modified to accept a single `input_value` parameter.

In [11]:
class Input(Instruction):
    def call(self, input_value = None):
        self.put_val(0, input_value)
        return self.return_control()

In [12]:
class Output(Instruction):
    def call(self):
        val = self.get_val(0)
        return self.return_control(ACTION_OUTPUT, val)

In [13]:
class JumpIfTrue(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if in_val:
            return self.return_control(ACTION_JUMP, out_val)
        return self.return_control()

In [14]:
class JumpIfFalse(Instruction):
    def call(self):
        in_val = self.get_val(0)
        out_val = self.get_val(1)
        if not in_val:
            return self.return_control(ACTION_JUMP, out_val)   
        return self.return_control()

In [15]:
class LessThan(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 < val1 else 0
        self.put_val(2, out_val)
        return self.return_control()

In [16]:
class Equals(Instruction):
    def call(self):
        val0 = self.get_val(0)
        val1 = self.get_val(1)
        out_val = 1 if val0 == val1 else 0
        self.put_val(2, out_val)
        return self.return_control()

In [25]:
class Halt(Instruction):
    def call(self):
        return self.return_control(ACTION_HALT)

In [26]:
OPS = {
    ADD: Add,
    MULTIPLY: Multiply,
    INPUT: Input,
    OUTPUT: Output,
    JUMP_IF_TRUE: JumpIfTrue,
    JUMP_IF_FALSE: JumpIfFalse,
    LESS_THAN: LessThan,
    EQUALS: Equals,
    HALT: Halt,
}

In [19]:
def create_instruction(memory, memory_pointer):
    encoded_opcode = memory[memory_pointer]
    opcode = int(str(encoded_opcode)[-2:])
    
    num_params = NUM_PARAMS[opcode]
    
    reverse_modes = str(encoded_opcode)[:-2][::-1]
    modes = [int(char) for char in reverse_modes] + [0] * (num_params - len(reverse_modes))
    
    params = memory[memory_pointer + 1 : memory_pointer + num_params + 1]
    
    return OPS[opcode](opcode, num_params + 1, params, modes, memory)

The `Program` class is modified slightly now too from how it was defined in Day 5. We now provide the `run` method with a list of `input_values`. Each time the `Program` encounters an `Input` instruction, it will take the next value from the `input_values` list as its input. (This requires that the caller knows in advance how many times an `Input` instruction should be encountered.)

We also add logic to deal with the `action` and `value` return tuple from the `Instruction.call` method. This allows flexibility for the `Program` to reset the `memory_pointer`, return a value to the caller, or simply halt.

In [20]:
class Program:
    def __init__(self, initial_memory):
        self.memory = initial_memory[:]
        self.memory_pointer = 0
        
    def run(self, input_values):
        input_counter = 0
        while(1):
            instruction = create_instruction(self.memory, self.memory_pointer)
                        
            if instruction.opcode == INPUT:
                action, value = instruction.call(input_values[input_counter])
                input_counter += 1
            else:
                action, value = instruction.call()
            
            if action == ACTION_NEXT:
                self.memory_pointer += instruction.length
            elif action == ACTION_JUMP:
                self.memory_pointer = value
            elif action == ACTION_HALT:
                break
            elif action == ACTION_OUTPUT:
                self.memory_pointer += instruction.length
                return value

## Part 1
With the above modifications to our classes, we define a helper function `run_amplifiers()`, which takes a phase sequence and runs each amplifier in order. The output of one amplifier is fed in as an input to the next amplifier.

In [21]:
def run_amplifiers(phase_sequence, data):
    input_signal = 0
    for phase in phase_sequence:
        program = Program(data)
        input_signal = program.run([phase, input_signal])
    return input_signal

To solve Part 1, we just iterate over all permutations of `[0,1,2,3,4]` and keep track of the highest final output of the amplifiers for each run.

In [22]:
max_signal = 0
for phase_sequence in itertools.permutations(range(5)):
    max_signal = max(max_signal, run_amplifiers(phase_sequence, data))
print(max_signal)

77500


## Part 2
For Part 2, we luckily don't have to change our class structure at all. In order to handle the looping paradigm for the amplifiers, we just need to make a modified version of `run_amplifiers` that has looping logic. This new function is called `loop_amplifiers`.

There were some tricy parts here:
* In order to not restart the amplifier at each new loop iteration, we have to store each amplifier as an instance of `Program`, and call `run()` on them repeatedly in subsequent loop iterations.
* In the first iteration of the loop, each amplifier receives 2 input values, just like in Part 1. However, in all subsequent iterations of the loop, each amplifier only receives 1 input value. That's because the `phase_sequence` value is only used on the first iteration of the loop.
* Since each amplifier is left running after it encounters its `Output` instruction, we need to make sure to advance the instruction pointer before pausing execution Otherwise, when we return to that amplifier we'll just repeat the same `Output` instruction over and over.

In [23]:
def loop_amplifiers(phase_sequence, data):
    input_signal = 0
    amplifier_index = 0
    is_first_loop = True
    amplifiers = []
    while(1):
        if is_first_loop:
            program = Program(data)
            amplifiers.append(program)
            program_input = [phase_sequence[amplifier_index], input_signal]
        else:
            program = amplifiers[amplifier_index]
            program_input = [input_signal]
         
        output_signal = program.run(program_input)
        if output_signal is None:
            return input_signal
        input_signal = output_signal
        amplifier_index = (amplifier_index + 1) % 5
        if amplifier_index == 0:
            is_first_loop = False

To get the answer for Part 2, we just do the same iteration over permutations, this time of `[5,6,7,8,9]`. Voila!

In [24]:
max_signal = 0
for phase_sequence in itertools.permutations([5,6,7,8,9]):
    max_signal = max(max_signal, loop_amplifiers(phase_sequence, data))
print(max_signal)

22476942
